In [ ]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get all stock info
print(msft.info)

# get historical market data
print(msft.history(period="1mo"))

# show meta information about the history (requires history() to be called first)
print(msft.history_metadata)

# show actions (dividends, splits, capital gains)
print(msft.actions)
print(msft.dividends)
print(msft.splits)
print(msft.capital_gains)  # only for mutual funds & etfs

# show share count
print(msft.get_shares_full(start="2022-01-01", end=None))

# show financials:
# - income statement
print(msft.income_stmt)
print(msft.quarterly_income_stmt)
# - balance sheet
print(msft.balance_sheet)
print(msft.quarterly_balance_sheet)
# - cash flow statement
print(msft.cashflow)
print(msft.quarterly_cashflow)
# see `Ticker.get_income_stmt()` for more options

# show holders
print(msft.major_holders)
print(msft.institutional_holders)
print(msft.mutualfund_holders)
print(msft.insider_transactions)
print(msft.insider_purchases)
print(msft.insider_roster_holders   )

# show recommendations
print(msft.recommendations)
print(msft.recommendations_summary)
print(msft.upgrades_downgrades      )

# Show future and historic earnings dates, returns at most next 4 quarters and last 8 quarters by default.
# Note: If more are needed use msft.get_earnings_dates(limit=XX) with increased limit argument.
print(msft.earnings_dates)

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
print(msft.isin)

# show options expirations
print(msft.options)

# show news
print(msft.news)

# get option chain for specific expiration
# opt = msft.option_chain('2024-03-01')
# data available via: opt.calls, opt.puts

Multiple tickers

In [ ]:

import yfinance as yf

tickers = yf.Tickers('msft aapl goog')

# access each ticker using (example)
tickers.tickers['MSFT'].info
tickers.tickers['AAPL'].history(period="1mo")
tickers.tickers['GOOG'].actions

To download price history into one table:

In [ ]:
import yfinance as yf
data = yf.download("SPY AAPL", period="1mo")

### smarter scraping
Install the nospam packages for smarter scraping using pip (see Installation). These packages help cache calls such that Yahoo is not spammed with requests.

To use a custom requests session, pass a session= argument to the Ticker constructor. This allows for caching calls to the API as well as a custom way to modify requests via the User-agent header.

In [ ]:
import requests_cache
session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'my-program/1.0'
ticker = yf.Ticker('msft', session=session)
# The scraped response will be stored in the cache
ticker.actions

In [ ]:
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

In [ ]:
import requests
import csv
import pandas as pd
from modules.db_sqlLite_utils import db_utils

api_key = 'WSCL1ZZL8PCWUGV9'
CSV_URL  = f"https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={api_key}"

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

# Assuming the first row contains column headers
column_headers = my_list[0]

# Creating a DataFrame excluding the header row
df = pd.DataFrame(my_list[1:], columns=column_headers)

# Displaying the DataFrame
print(df)
db_utils.create_db()
db_utils.insert_listing_status(df)

In [ ]:
import requests

def get_all_tickers():
    url = "https://finance.yahoo.com/screener/predefined/world_etfs"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        tickers = [item['symbol'] for item in data['finance']['result'][0]['quotes']]
        return tickers
    else:
        print("Failed to fetch data from Yahoo Finance API")
        return []

all_tickers = get_all_tickers()
print(all_tickers)

In [ ]:
exchange = 'NYSE'  # Replace 'NYSE' with any other exchange
exchange_tickers = db_utils.get_tickers_by_exchange(exchange)
print(len(exchange_tickers))